In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px

diabetes_data = pd.read_csv('data_2/diabetes_data.csv', sep=',')
#display(diabetes_data.info())
print(diabetes_data.shape)

#Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их.
# Для поиска используйте все признаки в данных. Сколько записей осталось в данных?
dupl_columns = list(diabetes_data.columns)

mask = diabetes_data.duplicated(subset=dupl_columns)
diabetes_data_duplicates = diabetes_data[mask]
#print(f'Число найденных дубликатов: {diabetes_data_duplicates.shape[0]}')

diabetes_duplicates = diabetes_data.drop_duplicates(subset=dupl_columns)
#print(f'Результирующее число записей: {diabetes_duplicates.shape[0]}')


#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes_duplicates.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes_duplicates[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes_duplicates[col].nunique() / diabetes_duplicates[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

#Итак, мы нашли шесть неинформативных признаков. Теперь можно удалить их с помощью метода drop(),
#передав результирующий список в его аргументы.
information_diabetes_data = diabetes_duplicates.drop(low_information_cols, axis=1)
#print(f'Результирующее число признаков: {information_diabetes_data.shape[1]}')
#display(information_diabetes_data.head())

#display(information_diabetes_data.isnull().tail())


list_sort = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for col in list_sort:
    information_diabetes_data[col] = information_diabetes_data[col].apply(lambda x: np.nan if x == 0 else x)
#display(information_diabetes_data.head())
#print(information_diabetes_data[information_diabetes_data['Glucose'] == 0])

cols_null_percent = round(information_diabetes_data.isnull().mean() * 100, 2)
#display(cols_null_percent)
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
#display(cols_with_null)


#создаем копию исходной таблицы
drop_diabetes_data = information_diabetes_data.copy()
#задаем минимальный порог: вычисляем 70% от числа строк
thresh = drop_diabetes_data.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
drop_diabetes_data = drop_diabetes_data.dropna(thresh=thresh, axis=1)
#удаляем записи, в которых есть хотя бы 2 пропускa
m = drop_diabetes_data.shape[1]
drop_diabetes_data = drop_diabetes_data.dropna(thresh=m-2, axis=0)
#отображаем результирующую долю пропусков
drop_diabetes_data.isnull().mean()
#print(drop_diabetes_data.shape)
display(drop_diabetes_data.head())



#В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.


#создаем копию исходной таблицы
fill_data = drop_diabetes_data.copy()
#создаем словарь имя столбца: число(признак) на который надо заменить пропуски
values = {
    'Pregnancies': fill_data['Pregnancies'].median(),
    'Glucose': fill_data['Glucose'].median(),
    'BloodPressure': fill_data['BloodPressure'].median(),
    'SkinThickness': fill_data['SkinThickness'].median(),
    'BMI': fill_data['BMI'].median(),
    'DiabetesPedigreeFunction': fill_data['DiabetesPedigreeFunction'].median(),
    'Age': fill_data['Age'].median(),
    'Outcome': fill_data['Outcome'].median(),
}
#заполняем пропуски в соответствии с заявленным словарем
fill_data = fill_data.fillna(values)
#выводим результирующую долю пропусков
fill_data.isnull().mean()

display(fill_data['SkinThickness'].mean())



#Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr(fill_data, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')





def outliers_iqr_log(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned


outliers, cleaned = outliers_iqr_log(fill_data, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

#Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?

def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned


outliers, cleaned = outliers_z_score(fill_data, 'SkinThickness')
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')


#Можно воспользоваться столбчатой диаграммой, чтобы визуально оценить соотношение числа пропусков к числу записей.
# Самый быстрый способ построить её — использовать метод plot():
#cols_with_null.plot(
#    kind='bar',
#    figsize=(10, 4),
#    title='Распределение пропусков в данных'
#);




(778, 10)
Gender: 100.0% одинаковых значений


,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98.0,58.0,33.0,34.0,0.430,43,0
1,2,112.0,75.0,32.0,35.7,0.148,21,0
2,2,108.0,64.0,NaN,30.8,0.158,21,0
3,8,107.0,80.0,NaN,24.6,0.856,34,0
4,7,136.0,90.0,NaN,29.9,0.210,50,0


np.float64(29.109067017082786)

Число выбросов по методу Тьюки: 29
Результирующее число записей: 732
Число выбросов по методу Тьюки: 0
Результирующее число записей: 761
Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757
